# Simple stochastics in python

Here, we take a look at moments, estimating distributions, correlation functions and power spectral densities.

In [2]:
%pylab
import sys
sys.path.append('..')
import scipy.signal as ssi
import scipy.special as ssp
import uit_scripts.stat_analysis.distribution as sad

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
def AR1(N, phi, X0, c=0, sigma=1):
    # The first-order autoregressive model. See https://en.wikipedia.org/wiki/Autoregressive_model
    eps = np.random.normal(scale=sigma,size=N)
    X = np.zeros(N)
    X[0]=X0
    for i in range(1,N):
        X[i] = c + phi*X[i-1] + eps[i]
        
    return X

Show some sample paths. If abs(phi)<1, the process is stationary.

In [8]:
def plot_X_paths():
    N = int(1e2)
    phi = -1.3
    X0 = 0.

    for i in range(5):
        X = AR1(N, phi, X0)
        plt.figure('paths')
        plt.plot(np.arange(N), X)

        print('mean value: {}, rms value: {}'.format(X.mean(), X.std()))
plot_X_paths()

mean value: 119457417.79828691, rms value: 3305468274.8447537
mean value: 1051835782.3184925, rms value: 29105013911.419254
mean value: -51956312.89352418, rms value: 1437666634.9722805
mean value: 16520801.86684565, rms value: 457141862.34962606
mean value: -1060579376.1160083, rms value: 29346955117.284725


# Estimate probability distribution

We use two methods: a simple histogram, as well as an empirical CDF

In [9]:
def est_dist():
    N = int(10000)
    phi = 0.3
    X0 = 0.
    
    X = AR1(N, phi, X0)
    
    Xn = (X-X.mean())/X.std()
    
    # Histogram
    hist, bin_edge = np.histogram(Xn, bins=64, density = True)
    bins = (bin_edge[1:]+bin_edge[:-1])/2
    
    # ECDF
    Xsort = np.sort(Xn)
    ECDF = np.linspace(0.,1.,Xn.size)
    
    plt.figure('histogram')
    plt.plot(bins, hist, 'o')
    plt.plot(bins, np.exp(-bins**2/2)/np.sqrt(2*np.pi), label='normal dist')
    plt.xlabel('X')
    plt.ylabel('p_X')
    plt.legend()
    
    plt.figure('ECDF')
    plt.plot(Xsort, ECDF)
    plt.plot(Xsort, 0.5*(1+ssp.erf(Xsort/np.sqrt(2))), label='normal dist')
    plt.xlabel('X')
    plt.ylabel('CDF_X')
    plt.legend()
    
est_dist()

# Estimate autocorrelation function

In [10]:
def est_corr():
    N = int(1000)
    phi = 0.6
    X0 = 0.
    
    X = AR1(N, phi, X0)
    
    Xn = (X-X.mean())/X.std()
    
    R = ssi.correlate(Xn, Xn, mode='full')
    n = np.arange(-(X.size-1), X.size)
    
    # Biased correlation function (introduces systematic errors)
    Rb = R/X.size
    # Unbiased correlation function (introduces no systematic errors, but diverges)
    Rub = R/(X.size-np.abs(n))
    
    Rtrue = phi**np.abs(n)
    
    plt.figure('AC')
    plt.plot(n, Rub, label='unbiased')
    plt.plot(n, Rb, label='biased')
    plt.plot(n, Rtrue, 'k:', label='true R')
    plt.xlabel('n')
    plt.ylabel('R_X(n)')
    plt.legend()
est_corr()

# Estimate power spectral density

In [14]:
def est_psd():
    N = 2**14
    phi = 0.6
    X0 = 0.
    
    X = AR1(N, phi, X0)
    
    Xn = (X-X.mean())/X.std()
    
    # Periodogram - simple calculation
    fp, P_Xn = ssi.periodogram(Xn, fs= 1.)
    
    plt.figure('psd - lin')
    plt.plot(fp, P_Xn, label='periodogram')
    
    plt.figure('psd - log')
    plt.loglog(fp, P_Xn, label='periodogram')
    
    # Using the Welch method - window size (nperseg) is most important for us.
    for nperseg in 2**np.array([13, 10, 7]):
        f, S_Xn = ssi.welch(Xn, fs=1., nperseg=nperseg)
        
        plt.figure('psd - lin')
        plt.plot(f[1:], S_Xn[1:], label='welch 2^{}'.format(int(np.log2(nperseg))))

        plt.figure('psd - log')
        plt.loglog(f[1:], S_Xn[1:], label='welch 2^{}'.format(int(np.log2(nperseg))))     
    
    plt.figure('psd - lin')
    plt.plot(fp, (1+phi**2-2*phi*np.cos(2*np.pi*fp))**(-1), 'k:', label='true')
    plt.xlabel('f')
    plt.ylabel('PSD')
    plt.legend()
    
    
    plt.figure('psd - log')   
    plt.loglog(fp, (1-phi**2)/(1+phi**2-2*phi*np.cos(2*np.pi*fp)), 'k:', label='true')
    plt.xlabel('f')
    plt.ylabel('PSD')
    plt.legend()
est_psd()

In [12]:
ssi.welch?

Signature:
ssi.welch(
    x,
    fs=1.0,
    window='hann',
    nperseg=None,
    noverlap=None,
    nfft=None,
    detrend='constant',
    return_onesided=True,
    scaling='density',
    axis=-1,
    average='mean',
)
Docstring:
Estimate power spectral density using Welch's method.

Welch's method [1]_ computes an estimate of the power spectral
density by dividing the data into overlapping segments, computing a
modified periodogram for each segment and averaging the
periodograms.

Parameters
----------
x : array_like
    Time series of measurement values
fs : float, optional
    Sampling frequency of the `x` time series. Defaults to 1.0.
window : str or tuple or array_like, optional
    Desired window to use. If `window` is a string or tuple, it is
    passed to `get_window` to generate the window values, which are
    DFT-even by default. See `get_window` for a list of windows and
    required parameters. If `window` is array_like it will be used
    directly as the window and its l